In [4]:
import mti
import nab
import numpy as np

In [2]:
mti.mti(np.array([0, 0, 1]), np.array([0, 0, 0]))

0.14285714285714285

In [5]:
nab.nab(np.array([0, 0, 1]), np.array([0, 0, 0]))

0.0990990990990991

In [3]:
tmp = mti.MTI()

tmp.compute_metrics(np.array([0, 0, 1]), np.array([0, 0, 0]), pos_label=1)

tmp.alarm_cardinality_score

0